[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/main/tutorials/feature-use/season_multiplicative_air_travel.ipynb)

# Multiplicative Seasonality
By default NeuralProphet fits additive seasonalities, meaning the effect of the seasonality is added to the trend to get the forecast. This time series of the number of air passengers is an example of when additive seasonality does not work:

In [1]:
if "google.colab" in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes

import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

set_log_level("ERROR")

In [2]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
df = pd.read_csv(data_location + "air_passengers.csv")

In [3]:
m = NeuralProphet()
metrics = m.fit(df, freq="MS")

Finding best initial lr:   0%|          | 0/209 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [4]:
future = m.make_future_dataframe(df, periods=50, n_historic_predictions=len(df))
forecast = m.predict(future)
m.plot(forecast)
# m.plot_parameters()

Predicting: 9it [00:00, ?it/s]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': 'rgba(45, 146, 255, 1.0)', 'width': 2},
              'mode': 'lines',
              'name': 'yhat1',
              'type': 'scatter',
              'uid': 'bb2af870-a9be-400c-a2b2-54a8bd5eace2',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
                       

This time series has a clear yearly cycle, but the seasonality in the forecast is too large at the start of the time series and too small at the end. In this time series, the seasonality is not a constant additive factor as assumed by NeuralProphet, rather it grows with the trend. This is multiplicative seasonality.

NeuralProphet can model multiplicative seasonality by setting `seasonality_mode="multiplicative"` in the input arguments:

In [5]:
m = NeuralProphet(seasonality_mode="multiplicative")
metrics = m.fit(df, freq="MS")

Finding best initial lr:   0%|          | 0/209 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

In [6]:
future = m.make_future_dataframe(df, periods=50, n_historic_predictions=len(df))
forecast = m.predict(future)
m.plot(forecast)
# m.plot_parameters()

Predicting: 9it [00:00, ?it/s]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': 'rgba(45, 146, 255, 1.0)', 'width': 2},
              'mode': 'lines',
              'name': 'yhat1',
              'type': 'scatter',
              'uid': '60d6a524-9caa-437f-a2b1-b007ef43a087',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
                       

The components figure will now show the seasonality as a percent of the trend:

In [7]:
m.plot_components(forecast)

FigureWidgetResampler({
    'data': [{'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': 'Trend',
              'showlegend': False,
              'type': 'scatter',
              'uid': '6c2a5405-4f7e-4a15-b98a-6e0dfe9c42aa',
              'x': array([datetime.datetime(1949, 1, 1, 0, 0),
                          datetime.datetime(1949, 2, 1, 0, 0),
                          datetime.datetime(1949, 3, 1, 0, 0),
                          datetime.datetime(1949, 4, 1, 0, 0),
                          datetime.datetime(1949, 5, 1, 0, 0),
                          datetime.datetime(1949, 6, 1, 0, 0),
                          datetime.datetime(1949, 7, 1, 0, 0),
                          datetime.datetime(1949, 8, 1, 0, 0),
                          datetime.datetime(1949, 9, 1, 0, 0),
                          datetime.datetime(1949, 10, 1, 0, 0),
                          datetime.datetime(1949, 11, 1, 0, 0),
                          datetime

Note that the seasonality is only fit on data occuring at the start of the month. Thus, the plotted values for seasonality inbetween months may take on random values.

Setting`seasonality_mode="multiplicative"` will model all seasonalities as multiplicative, including custom seasonalities added with `add_seasonality`.